In [1]:
!nvidia-smi

Tue Dec 10 09:39:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Init

In [2]:
!pip install transformers

!pip install datasets

!pip install sacrebleu

!pip install torch>=1.13.0

!pip install fsspec==2024.9.0

!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.4 MB/s eta 0:00:00
 

In [3]:
import random
random.seed(120)

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


## Load Datasets

In [6]:
BASE_PATH = "/content/gdrive/MyDrive/experiments"
TRAIN_UNRESTRICTED_PATH = f"{BASE_PATH}/unrestricted_train_dataset-50.csv"
TRAIN_RESTRICTED_PATH = f"{BASE_PATH}/restricted_train_dataset.csv"
OOV_UNRESTRICTED_PATH = f"{BASE_PATH}/unrestricted_test_dataset.csv"
OOV_RESTRICTED_PATH = f"{BASE_PATH}/restricted_test_dataset.csv"

In [7]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets


unrestricted_train = load_dataset('csv', data_files=TRAIN_UNRESTRICTED_PATH)
restricted_train = load_dataset('csv', data_files=TRAIN_RESTRICTED_PATH)
unrestricted_test = load_dataset('csv', data_files=OOV_UNRESTRICTED_PATH)
restricted_test = load_dataset('csv', data_files=OOV_RESTRICTED_PATH)

# adding label column based if its restricted or not
unrestricted_train['train'] = unrestricted_train['train'].add_column('label', [0] * len(unrestricted_train['train']))
unrestricted_test['train'] = unrestricted_test['train'].add_column('label', [0] * len(unrestricted_test['train']))
restricted_train['train'] = restricted_train['train'].add_column('label', [1] * len(restricted_train['train']))
restricted_test['train'] = restricted_test['train'].add_column('label', [1] * len(restricted_test['train']))

#mixed
train_dataset = concatenate_datasets([unrestricted_train['train'], restricted_train['train']]).shuffle()
test_dataset = concatenate_datasets([unrestricted_test['train'], restricted_test['train']]).shuffle()

dataset = DatasetDict({'train': train_dataset,'test': test_dataset})

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label'],
        num_rows: 15000
    })
})

## Preprocessing

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #tokenization model, smaller version of BERT

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["en"], truncation=True) # from text to token

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True) #applied preprocess_function in each dataset value

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [11]:
item = tokenized_dataset['train']['en'][0]
decoded_item = tokenizer.decode(tokenized_dataset['train']['input_ids'][0]) # from token to text

reversed_vocab = {i: w for w, i in tokenizer.get_vocab().items()} # map input_ids with original token

print(item)
print(decoded_item)
#Compare the number of words in the original text (item), the number of tokens (input_ids), and the number of words in the decoded text (decoded_item)
print(len(item.split(' ')), len(tokenized_dataset['train']['input_ids'][0]), len(decoded_item.split(' ')))
#Shows the numeric indexes of the tokenized sequence for the first example.
print(tokenized_dataset['train']['input_ids'][0])
#Converts numeric indexes into readable tokens using reverse vocabulary, to match the actual tokens.
print([reversed_vocab[i] for i in tokenized_dataset['train']['input_ids'][0]])

every time both an elevator falls and the brake is pressed then at some point a motorbike falls down or a constructor creates instances
[CLS] every time both an elevator falls and the brake is pressed then at some point a motorbike falls down or a constructor creates instances [SEP]
24 29 26
[101, 2296, 2051, 2119, 2019, 7764, 4212, 1998, 1996, 13428, 2003, 4508, 2059, 2012, 2070, 2391, 1037, 5013, 5638, 3489, 4212, 2091, 2030, 1037, 9570, 2953, 9005, 12107, 102]
['[CLS]', 'every', 'time', 'both', 'an', 'elevator', 'falls', 'and', 'the', 'brake', 'is', 'pressed', 'then', 'at', 'some', 'point', 'a', 'motor', '##bi', '##ke', 'falls', 'down', 'or', 'a', 'construct', '##or', 'creates', 'instances', '[SEP]']


In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label', 'input_ids', 'attention_mask'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label', 'input_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [13]:
tokenized_dataset['train']['label'][49000:49010]

[0, 1, 0, 0, 1, 0, 1, 0, 0, 0]

## Training

training a classifier based on a pre-trained DistilBERT model for a binary classification task.

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#Loads the pre-trained DistilBERT model and specifies that there are 2 output labels (binary classification).
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2) #fine tuning, AutoModelForSequenceClassification: Loads a pre-trained model (in this case, distilbert-base-uncased) and configures it for classification.

model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 768, padding_idx=0)

In [16]:
print("For T5:")
print("Tokenizer vocab_size: {}".format(tokenizer.vocab_size))
print("Model vocab size: {}\n".format(model.config.vocab_size))

For T5:
Tokenizer vocab_size: 30522
Model vocab size: 30522



In [17]:
RESULTS_DIR = "/content/gdrive/MyDrive/experiments/classifier"

STEPS = 31

training_args = TrainingArguments(
    output_dir=RESULTS_DIR,
    evaluation_strategy="steps",
    eval_steps=STEPS,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    # num_train_epochs=1,
    max_steps=STEPS,
    adam_beta2=0.98,
    warmup_steps=100,
    optim="adamw_torch",
    fp16=True,
    save_strategy='steps',
    save_steps=STEPS-1
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
import numpy as np
import evaluate
from sklearn.metrics import precision_recall_fscore_support

metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    preds = eval_preds.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = metric.compute(references=labels, predictions=preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'preds': preds.tolist(),
        'labels': labels.tolist()
    }

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

#  trainer.train("/content/gdrive/MyDrive/final_thesis_experiments/checkpoint-5625/")
trainer.train()

<ipython-input-25-6ca5b950cea3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


Trainer is attempting to log a value of "{'accuracy': 1.0}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1

TrainOutput(global_step=31, training_loss=0.4827819331999748, metrics={'train_runtime': 22.2935, 'train_samples_per_second': 22.249, 'train_steps_per_second': 1.391, 'total_flos': 5021342330304.0, 'train_loss': 0.4827819331999748, 'epoch': 0.00496})

## Test

In [20]:
trainer.evaluate(tokenized_dataset["test"])

Trainer is attempting to log a value of "{'accuracy': 0.9431333333333334}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1

{'eval_loss': 0.5770681500434875,
 'eval_accuracy': {'accuracy': 0.9431333333333334},
 'eval_f1': 0.9461931495615972,
 'eval_precision': 0.8978810008380222,
 'eval_recall': 1.0,
 'eval_preds': [1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1

In [21]:
from transformers import DistilBertForSequenceClassification

In [22]:
list(tokenized_dataset['test']['en'][:300])

['at any time both sxsvmjw_r and tshnfptj',
 'all the time either the plane takes off, the plane lands or a computer starts',
 'all the time m_sjzWjWN',
 'always when W_ZrjJUwKxYYR then sooner or later GlVOjhrADOqffM and oOYihxCtE',
 'if either bIJEBuJucRfwFJ or tQXIX then it will happen that both pFeVvjUk and wTRMtDXljs',
 'sooner or later MUMxkzbEby and pJZEsTQPZqld after niaJLORUu or TefQLo',
 'as the model underfits and a cpu cools down, at some point the cpu is cooled down or a fridge is switched on',
 'in any case the cpu overheats or a fridge is empty or the monitor shows output',
 'every time BDHvKg or sPRNVd then at some point in time ZUBErALXQKCmMTT',
 'forever, either a cpu cools down, a fridge is switched on or the model overfits',
 'in the future ikVOJCofeCWmcH after either uspoVPftwsSsZj or FtgWDH',
 'it will not happen that mUNOQrVrZb and, at the same time, khxYkLCxN and rOT_L',
 'each time a model overfits then whenever a cpu is cooled down then at a certain moment a mo

In [26]:
from transformers import AutoModelForSequenceClassification

# Load the model using the same architecture as training
prediction_model = AutoModelForSequenceClassification.from_pretrained(
    "/content/gdrive/MyDrive/experiments/classifier/checkpoint-31/",
    num_labels=2
)

# Get your test data
src_text = tokenized_dataset['test']['en'][2500:2600]

# Tokenize the input texts
tokens = tokenizer(src_text, return_tensors="pt", padding=True)

# Print the source texts and tokens for debugging
print("Source texts:")
print(src_text)
print("\nTokenized inputs:")
print(tokens)

# Get predictions
outputs = prediction_model(**tokens)
predictions = outputs.logits.argmax(-1)

# If you need probabilities
import torch
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Print predictions
print("\nPredictions:")
print(predictions)
print("\nProbabilities:")
print(probabilities)

Source texts:
['it is going to happen that mjSFCQXLs_OpzMg, LTncvP and SxpUgOW', 'always, _bjnJDBZb and, at the same time, CJRhnyWU and OLPKi', 'whenever LYHYRBwtSXfvp then at some point in time GFXfX', 'whenever PlFoSqw and oRFLTfaTNHR then sooner or later AEzFyE and ywfOvffSjY', 'at a certain moment the model is underfitted and the model is overfitted after either a cpu is cooled down or a fridge is switched on', 'sooner or later laQZBIZwiWYUv after dRhgD', 'it is always true that both CvuyhIPVpRsdTb and AkIZF', 'each time zKvzGtcq then whenever beoauDT_Vwpc_XY then in the future VWRtpaDqBdvJNsV', 'every time both lTUpvMuanDXqV and Bwi_JtZHkSY', 'always when the mouse captures input and the model underfits then at some point either the monitor lights up or a plane takes off', 'it is always the case that rxpuUcDcpwA', 'in the future the computer stops or a model is overfitted or a fridge is open', 'forever, a cpu overheats and, at the same time, the model is overfitted and a plane lan